Coding Challenges: Chapter 7
Nishan Karki
BINF 3360
Computational Biology
Date: 04/29/2022

# 7.3 Step 11

In [4]:
import numpy as np

def limbLength(n, j, dist_m):
#         print('n:',n)
#         print('j:',j)
#         print(dist_m)
                
        dist_l = []    
        for i in range(n):
            for k in range(n):
                if j!=k and k!=i and i!=j:
                    leaves_l = (dist_m[i][j]+dist_m[j][k]-dist_m[i][k])/2
                    
                    dist_l.append(leaves_l)
        return int(min(dist_l))
            

def main():
    
    with open('/Users/nises/Downloads/dataset_604515_11.txt') as input_data:
        read_lines = input_data.read().splitlines()
        n = int(read_lines[0])
        j = int(read_lines[1])

        distance_matrix = np.fromstring(read_lines[2], dtype=int, sep=' ')
        for line in read_lines[3:]:
            row = np.fromstring(line, dtype=int, sep=' ')
            distance_matrix = np.vstack([distance_matrix,row])
        
        print(limbLength(n, j, distance_matrix))

if __name__ == '__main__':
    main()

859


# 7.4 Step 6

In [6]:
def nearest(edge, weight, x, i, k):
    queue = [[i]]
    visited = set([i])
    findPath = []
    dist = 0
    while len(queue) > 0:
        path = queue.pop()
        node = path[-1]
        visited.add(node)
        if node == k:
            findPath = path
            break
        for next_node in edge[node]:
            if next_node not in visited:
                queue.append(path+[next_node])
    
    for k in range(len(findPath)-1):
        i, j = findPath[k], findPath[k+1]
        if dist+weight[(i, j)] > x:
            return (i, j, x-dist, dist+weight[(i, j)]-x)
        dist += weight[(i, j)]

In [ ]:
# AdditivePhylogeny(D)
#     n ← number of rows in D
#     if n = 2
#         return the tree consisting of a single edge of length D1,2
#     limbLength ← Limb(D, n)
#     for j ← 1 to n - 1
#         Dj,n ← Dj,n - limbLength
#         Dn,j ← Dj,n
#     (i, k) ← two leaves such that Di,k = Di,n + Dn,k
#     x ← Di,n
#     D ← D﻿ with row n and column n removed
#     T ← AdditivePhylogeny(D)
#     v ← the (potentially new) node in T at distance x from i on the path between i and k
#     add leaf n back to T by creating a limb (v, n) of length limbLength
#     return T

In [20]:
import numpy as np

def additivePhylogeny(matrix, n, inner_n):
    if n == 2:
        edge = {}
        edge[0] = [1]
        edge[1] = [0]
        weight = {}
        weight[(0, 1)] = matrix[0, 1]
        weight[(1, 0)] = matrix[0, 1]
        return (edge, weight, inner_n)

    limb = limbLength(n, n-1, matrix)
    matrix[:-1,-1] -= limb
    matrix[-1,:-1] -= limb
    
    for k1 in range(matrix.shape[0]-1):
        arr = matrix[k1] - matrix[-1]
        index = np.where(arr == matrix[k1, -1])
        if len(index[0]) > 0:
            i=index[0][0]
            k = k1
    
    x = matrix[i, -1]
    edge, weight, inner_n = additivePhylogeny(matrix[:-1, :-1], n-1, inner_n)
    
    i_near, k_near, i_x, n_x = nearest(edge, weight, x, i, k)
    new_node = i_near
    
    if i_x != 0:
        new_node = inner_n
        inner_n += 1
        edge[i_near].remove(k_near)
        edge[k_near].remove(i_near)
        edge[i_near].append(new_node)
        edge[k_near].append(new_node)
        edge[new_node] = [i_near, k_near]

        weight[(new_node, i_near)] = i_x
        weight[(i_near, new_node)] = i_x
        weight[(new_node, k_near)] = n_x
        weight[(k_near, new_node)] = n_x
        del weight[(i_near, k_near)]
        del weight[(k_near, i_near)]
        
    edge[new_node].append(n-1)
    edge[n-1] = [new_node]
    weight[(n-1, new_node)] = limb
    weight[(new_node, n-1)] = limb
    return (edge, weight, inner_n)

            
def main():
    
    with open('/Users/nises/Downloads/dataset_604516_6.txt') as input_data:
        read_lines = input_data.read().splitlines()
        n = int(read_lines[0])

        distance_matrix = np.fromstring(read_lines[1], dtype=int, sep=' ')
        for line in read_lines[2:]:
            row = np.fromstring(line, dtype=int, sep=' ')
            distance_matrix = np.vstack([distance_matrix,row])
            
        edge, weight, n = additivePhylogeny(distance_matrix, n, n)
        
        for i in sorted(edge):
            for j in sorted(edge[i]):
                print (str(i)+"->"+str(j)+":"+str(weight[(i, j)]))

if __name__ == '__main__':
    main()

0->21:516
1->20:363
2->23:468
3->22:170
4->37:244
5->34:774
6->24:845
7->25:835
8->26:529
9->27:924
10->28:427
11->29:206
12->30:702
13->31:459
14->32:791
15->33:156
16->34:155
17->35:842
18->36:342
19->37:704
20->1:363
20->25:532
20->27:418
21->0:516
21->27:875
21->31:792
22->3:170
22->31:89
22->33:648
23->2:468
23->30:875
23->34:522
24->6:845
24->29:584
24->36:520
25->7:835
25->20:532
25->29:922
26->8:529
26->28:119
26->32:317
27->9:924
27->20:418
27->21:875
28->10:427
28->26:119
28->35:582
29->11:206
29->24:584
29->25:922
30->12:702
30->23:875
30->36:733
31->13:459
31->21:792
31->22:89
32->14:791
32->26:317
32->33:658
33->15:156
33->22:648
33->32:658
34->5:774
34->16:155
34->23:522
35->17:842
35->28:582
35->37:520
36->18:342
36->24:520
36->30:733
37->4:244
37->19:704
37->35:520


# 7.4 Step 6

In [25]:
def neighborJoining(matrix, n):
    if n == 2:
        edge = {}
        weight = {}
        for i in matrix:
            for j in matrix[i]:
                if  j == i:
                    continue
                edge[i] = [j]
                edge[j] = [i]
                weight[(i, j)] = matrix[i][j]
                weight[(j, i)] = matrix[i][j]
        return (edge, weight)
    
    totalDistance = {}
    for i in matrix:
        totalDistance[i] = 0
        for j in matrix[i]:
            totalDistance[i] += matrix[i][j]
    MatrixD = {}
    for i in matrix:
        MatrixD[i] = {}
        for j in matrix[i]:
            if i == j:
                MatrixD[i][j] = 0
            else:
                MatrixD[i][j] = (n-2)*matrix[i][j] - totalDistance[i] - totalDistance[j]
                
    mini = float("inf")
    _i = _j = None
    for i in MatrixD:
        for j in MatrixD[i]:
            if MatrixD[i][j] < mini:
                mini = MatrixD[i][j]
                _i, _j = i, j
                
    delta = (totalDistance[_i] - totalDistance[_j])/(n-2.0)
    limbLength_i = (matrix[_i][_j] + delta)/2.0
    limbLength_j = (matrix[_i][_j] - delta)/2.0
    
    rows = list(matrix)
    m = max(matrix) + 1
    matrix[m] = {}
    matrix[m][m] = 0.0
    for k in rows:
        if k != _i and k != _j:
            matrix[m][k] = (matrix[_i][k] + matrix[_j][k] - matrix[_i][_j])/2.0
            matrix[k][m] = matrix[m][k]
            del matrix[k][_i]
            del matrix[k][_j]
    del matrix[_i]
    del matrix[_j]
    
    edge, weight = neighborJoining(matrix, n-1)\
    
    edge[_i] = [m]
    edge[m].append(_i)
    edge[_j] = [m]
    edge[m].append(_j)
    weight[(_i, m)] = limbLength_i
    weight[(m, _i)] = limbLength_i
    weight[(_j, m)] = limbLength_j
    weight[(m, _j)] = limbLength_j
    return edge, weight

def main():
    
    with open('/Users/nises/Downloads/dataset_604519_7.txt') as f:
        for i, line in enumerate(f):
            line = line.strip()
            if i == 0:
                n = int(line)
                matrix = {}
            else:
                row = line.split()
                matrix[i-1] = {}
                for j in range(len(row)):
                    matrix[i-1][j] = float(row[j])
            
        edge, weight = neighborJoining(matrix, n)
        for i in sorted(edge):
            for j in sorted(edge[i]):
                print (str(i)+"->"+str(j)+":"+str(weight[(i, j)]))
                
        

if __name__ == '__main__':
    main()

0->34:585.8303571428571
1->49:576.8413461538462
2->44:528.8368055555555
3->40:541.6875
4->45:518.9852941176471
5->35:572.4166666666666
6->43:516.9868421052631
7->37:571.865
8->36:616.1346153846154
9->40:542.3125
10->45:559.0147058823529
11->49:489.15865384615387
12->33:525.3620689655172
13->46:586.3203125
14->42:588.39375
15->32:580.9833333333333
16->34:547.1696428571429
17->42:528.60625
18->50:522.3333333333334
19->35:519.5833333333334
20->41:581.8452380952381
21->38:604.5
22->36:481.86538461538464
23->50:579.6666666666666
24->43:507.0131578947368
25->47:509.41875
26->33:533.6379310344828
27->39:478.8369565217391
28->51:541.8607954545455
29->53:672.1935763888889
30->32:512.0166666666667
31->39:607.1630434782609
32->15:580.9833333333333
32->30:512.0166666666667
32->44:130.16319444444446
33->12:525.3620689655172
33->26:533.6379310344828
33->37:75.63499999999999
34->0:585.8303571428571
34->16:547.1696428571429
34->38:106.5
35->5:572.4166666666666
35->19:519.5833333333334
35->51:140.63920

# 7.9 Step 10

In [9]:
import copy
import numpy as np

def hamming(s1,s2):
    assert len(s1) == len(s2)
    d = 0
    for i in range(len(s1)):
        if s1[i]!=s2[i]:
            d+=1
    return d

In [14]:
# SmallParsimony(T, Character)
#     for each node v in tree T
#         Tag(v) ← 0
#         if v is a leaf
#             Tag(v) ← 1
#             for each symbol k in the alphabet
#                 if Character(v) = k
#                     sk(v) ← 0
#                 else
#                     sk(v) ← ∞
#     while there exist ripe nodes in T
#         v ← a ripe node in T
#         Tag(v) ← 1
#         for each symbol k in the alphabet
#             sk(v) ← minimumall symbols i {si(Daughter(v))+αi,k} + minimumall symbols j {sj(Son(v))+αj,k}
#     return minimum over all symbols k {sk(v)}

In [11]:
def small_parsimony_problem(n,edges,labels):
    l = len(labels[0])
    alphabet = sorted(list(set(''.join(labels))))
    k = len(alphabet)
    m = 2*n-1  
    
    d = dict(zip(alphabet,range(k)))
    s = [[' ']*l for i in range(m)]
    sk = np.ndarray(shape=(m,k,l), dtype=float)
    sk.fill((m-1)*l)

    tree = {}
    parent = {}
    for edge in edges:
        node = edge[0]
        child = edge[1]
        tree.setdefault(node,[]).append(child)
        parent[child] = node

    root = parent[list(parent.keys())[0]]
    while root in parent.keys():
        root = parent[root]

    for i,label in enumerate(labels):
        s[i] = list(label)
        for j,c in enumerate(label):
            sk[i,d[c],j] = 0
    
    for i in range(l):
        def dfs_sk(node):
            if node < n:
                return
            lnode = tree[node][0]
            rnode = tree[node][1]
            dfs_sk(lnode)
            dfs_sk(rnode)
            for j in range(k):
                mask = np.ones(k)
                mask[j] = 0
                sk[node,j,i] = min(sk[lnode,:,i]+mask) + min(sk[rnode,:,i]+mask)
            return
        dfs_sk(root)
        
    parsimony = sum(sk[root].min(axis=0))
    
    for i in range(l):        
        def dfs_s(node):
            if node < n:
                return
            c = sk[node,:,i]
            if node == root:                
                s[node][i] = alphabet[c.argmin()]
            else:
                pnode = parent[node]
                j = d[s[pnode][i]]
                mask = np.ones(k)
                mask[j] = 0
                c += mask
                s[node][i] = alphabet[c.argmin()]
            
            lnode = tree[node][0]
            rnode = tree[node][1]
            dfs_s(lnode)
            dfs_s(rnode)
        dfs_s(root)
    
    ret = []
    for node,(lnode,rnode) in tree.items():
        ps = ''.join(s[node])
        ls = ''.join(s[lnode])
        rs = ''.join(s[rnode])
        ret.append((ps,ls))
        ret.append((ps,rs))
        
    lbranch = (''.join(s[root]), ''.join(s[tree[node][0]]))
    rbranch = (''.join(s[root]), ''.join(s[tree[node][1]]))
    return (parsimony, ret[:],rbranch,lbranch)

def main():
    
    with open('/Users/nises/Downloads/dataset_604521_10.txt') as f:
        lines = f.read().strip().split('\n')
        n = int(lines[0])
        labels = []
        edges = []
        for l in lines[1:]:
            a = l.split('->')[0]
            b = l.split('->')[1]
            if a.isdigit() == True:
                a = int(a)
            else:
                if a not in labels:
                    labels.append(a)
                a = labels.index(a)
            if b.isdigit() == True:
                b = int(b)
            else:
                if b not in labels:
                    labels.append(b)
                b = labels.index(b)
            edges.append((a,b))
                
#         print(n)
#         print(edges)
#         print(labels)

        parsimony, pes, r, l = small_parsimony_problem(n,edges,labels)
        print(parsimony)

        for a in pes:
            print(a[0]+'->'+a[1]+':'+str(hamming(a[0],a[1])))
            print(a[1]+'->'+a[0]+':'+str(hamming(a[1],a[0])))
        

if __name__ == '__main__':
    main()

10909.0
CCAGCAAACCATATGCAGAATCCCGGCAGGAGTACACGAAGACTAGGCAACGTCGCTAAAAAACTGGTAGCAATATGATAGCCCTCAGGAAAGTGCCAATTACTTTCCCGCGCCATGGATCTCAAGTCGGCATGTATTAAATGAGGTGGTCAAA->CCGGTATATCGTATGGAAAAACCCGGCCGGAGGACAGGGAATGTGGTCATCCTTGCTCGAACACTCTTACATATGTGATAGCTCTCAGGTAGTGGTTCGTTTCTTTCTCGCAGCCTTGATCTCTGGCGGTCTTGGTTTAAGTGTGTGACAAACA:61
CCGGTATATCGTATGGAAAAACCCGGCCGGAGGACAGGGAATGTGGTCATCCTTGCTCGAACACTCTTACATATGTGATAGCTCTCAGGTAGTGGTTCGTTTCTTTCTCGCAGCCTTGATCTCTGGCGGTCTTGGTTTAAGTGTGTGACAAACA->CCAGCAAACCATATGCAGAATCCCGGCAGGAGTACACGAAGACTAGGCAACGTCGCTAAAAAACTGGTAGCAATATGATAGCCCTCAGGAAAGTGCCAATTACTTTCCCGCGCCATGGATCTCAAGTCGGCATGTATTAAATGAGGTGGTCAAA:61
CCAGCAAACCATATGCAGAATCCCGGCAGGAGTACACGAAGACTAGGCAACGTCGCTAAAAAACTGGTAGCAATATGATAGCCCTCAGGAAAGTGCCAATTACTTTCCCGCGCCATGGATCTCAAGTCGGCATGTATTAAATGAGGTGGTCAAA->TGAACTAACCATATGTGGTATGCCCGCAGAGCTTATCAAAGACGTGGACCGGTCGTAACGCATACGGTAGCACAACGTTGCTCTTGGACATAGTGCCAATTATTTTACGGGGCCATGGATTTGGAGTCGGACTGTAGGTCACGAGGTGGTCGAC:59
TGAACTAACCATATGTGGTATGCCCGCAGAGCTTATCAAAGACGTGGACC

# 7.9 Step 12

In [15]:
def small_parsimony_unrooted_problem(n,uedges,labels):
    utree = {}
    m = n+1
    for uedge in uedges:
        node = uedge[0]
        child = uedge[1]
        m = max(node+1,child+1,m)
        utree.setdefault(node,[]).append(child)
    
    def get_rooted_tree(uedge):
        (lnode,rnode) = uedge
        root = m
        visited = [False]*(m+1)
        edges = []
        rooted_tree = copy.deepcopy(utree)
        rooted_tree[rnode].remove(lnode)
        rooted_tree[lnode].remove(rnode)
        rooted_tree[root] = [lnode,rnode]
        def dfs_rooted_tree(node):
            if (node < n) or (visited[node] == True):
                return
            visited[node] = True
            for child in rooted_tree[node]:
                if (visited[child] == False ):
                    edges.append((node,child))
                    dfs_rooted_tree(child)
            return
        dfs_rooted_tree(root)      
        return edges

    perls = []
    for uedge in set([(min(a,b),max(a,b)) for (a,b) in uedges]):
        edges = get_rooted_tree(uedge)
        perl = small_parsimony_problem(n,edges,labels)
        perls.append(perl)
    (p,e,r,l) = min(perls)
    ret = [(edge[0],edge[1]) for edge in e if (edge != r and edge !=l)]
    ret.append((l[1],r[1]))
    print(p)
    return (ret)

def main():
    
    with open('/Users/nises/Downloads/dataset_604521_12.txt') as f:

        lines = f.read().strip().split('\n')
        n = int(lines[0])
        labels = []
        edges = []
        for l in lines[1:]:
            a = l.split('->')[0]
            b = l.split('->')[1]
            if a.isdigit() == True:
                a = int(a)
            else:
                if a not in labels:
                    labels.append(a)
                a = labels.index(a)
            if b.isdigit() == True:
                b = int(b)
            else:
                if b not in labels:
                    labels.append(b)
                b = labels.index(b)
            edges.append((a,b))
        
#         print(n)
#         print(edges)
#         print(labels)
        
        pes = small_parsimony_unrooted_problem(n,edges,labels)

        for a in pes:
            print(a[0]+'->'+a[1]+':'+str(hamming(a[0],a[1])))
            print(a[1]+'->'+a[0]+':'+str(hamming(a[1],a[0])))
        

if __name__ == '__main__':
    main()

614.0
AAACATACTAAAGCCCCATGAAACCATAAAACAAA->AAAGATTGTAAAGCCCCATGAAACCATAAAACAAA:3
AAAGATTGTAAAGCCCCATGAAACCATAAAACAAA->AAACATACTAAAGCCCCATGAAACCATAAAACAAA:3
AAACATACTAAAGCCCCATGAAACCATAAAACAAA->AAACATACTACAGCCCCATGGAACCATAAAACAAA:2
AAACATACTACAGCCCCATGGAACCATAAAACAAA->AAACATACTAAAGCCCCATGAAACCATAAAACAAA:2
AAAGATTGTAAAGCCCCATGAAACCATAAAACAAA->ACAGCTTGGTACGCCCGAGGTTCGGAGATTAATAT:18
ACAGCTTGGTACGCCCGAGGTTCGGAGATTAATAT->AAAGATTGTAAAGCCCCATGAAACCATAAAACAAA:18
AAAGATTGTAAAGCCCCATGAAACCATAAAACAAA->TAAGAATGTAAATGAGCTTCAAACCTTTAATCAGC:13
TAAGAATGTAAATGAGCTTCAAACCTTTAATCAGC->AAAGATTGTAAAGCCCCATGAAACCATAAAACAAA:13
AAACATACTACAGCCCCATGGAACCATAAAACAAA->AAATATACTACACTCCCATGGCAACATGAAACGAG:8
AAATATACTACACTCCCATGGCAACATGAAACGAG->AAACATACTACAGCCCCATGGAACCATAAAACAAA:8
AAACATACTACAGCCCCATGGAACCATAAAACAAA->AAACGTACTACAGCCACATAGAACCATAAAACAAA:3
AAACGTACTACAGCCACATAGAACCATAAAACAAA->AAACATACTACAGCCCCATGGAACCATAAAACAAA:3
AAATATACTACACTCCCATGGCAACATGAAACGAG->AGCTGTACAACCCTCCGTGGGCAAAACGGGATGTG:14
AGCTGTACAACCCT

# END